In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../../tribe_dynamics_data.csv')
df.head(1)

,Unnamed: 0,Unnamed: 0.1,brand_id,worker_id,mturker,post_hash,answer,date,duration_seconds,text,model_decision,timestamped_model,lang
0,0,0,18792,TRIBE_103_allayne.low@gmail.com,False,3ca62dcea583b7aa204fc52fe6b2826c,False,2017-07-16,NaN,・\r\nケイト スペード ニューヨークの\r\n2017Fall Collectionに招...,False,NaN,ja
